Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# My Work

## Extra

In [0]:
# Make df['SALE_DATE'] to a more usable datetime method
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])

In [18]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,2019-01-01
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,2019-01-01
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,2019-01-01
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,2019-01-01
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,2019-01-01


In [19]:
df.describe(include='all')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
count,23040,23040,23040,23039,23040.000000,23040.000000,0.0,23039,23040,5201,23039.000000,23039.000000,23039.000000,23039.000000,22987,2.303900e+04,23005.000000,23040.000000,23040,2.304000e+04,23040
unique,5,11,44,10,NaN,NaN,NaN,146,22691,1724,NaN,NaN,NaN,NaN,3652,NaN,NaN,NaN,147,NaN,120
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,NaN,NaN,NaN,D4,100 JEROME STREET,4,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,D4,NaN,2019-01-24 00:00:00
freq,7494,19117,5061,11071,NaN,NaN,NaN,3408,6,99,NaN,NaN,NaN,NaN,7500,NaN,NaN,NaN,3408,NaN,480
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-30 00:00:00
mean,NaN,NaN,NaN,NaN,4459.116493,353.948481,NaN,NaN,NaN,NaN,10791.186987,1.827423,0.286514,2.320891,NaN,3.508555e+03,1819.671724,1.624479,NaN,1.328133e+06,NaN
std,NaN,NaN,NaN,NaN,3713.269887,628.025255,NaN,NaN,NaN,NaN,1103.414210,10.090432,5.778224,12.070554,NaN,2.218610e+04,488.376571,0.803898,NaN,1.025394e+07,NaN
min,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,-148.000000,0.000000,NaN,0.000000e+00,0.000000,1.000000,NaN,0.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,1341.000000,22.000000,NaN,NaN,NaN,NaN,10306.000000,0.000000,0.000000,1.000000,NaN,4.940000e+02,1920.000000,1.000000,NaN,0.000000e+00,NaN


In [20]:
df.shape

(23040, 21)

## Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [0]:
df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']

In [22]:
df.shape

(5061, 21)

##Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [23]:
start_date = '2019-01-01'
end_date = '2019-03-31'
traindate = (df['SALE_DATE'] >= start_date) & (df['SALE_DATE'] <= end_date)
train = df.loc[traindate]
print(train.shape)
train.head()

(4094, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,2019-01-01
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,2019-01-01
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01


In [24]:
start_date = '2019-04-01'
end_date = '2019-04-30'
testdate = (df['SALE_DATE'] >= start_date) & (df['SALE_DATE'] <= end_date)
test = df.loc[testdate]
print(test.shape)
test.head()

(967, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18182,1,OTHER,01 ONE FAMILY DWELLINGS,1,1708,24,NaN,A9,271 PLEASANT AVENUE,NaN,10029.0,1.0,0.0,1.0,928,2501.0,1890.0,1,A9,1,2019-04-01
18235,2,OTHER,01 ONE FAMILY DWELLINGS,1,5913,878,NaN,A1,4616 INDEPENDENCE AVENUE,NaN,10471.0,1.0,0.0,1.0,"5,000",2272.0,1930.0,1,A1,895000,2019-04-01
18239,2,OTHER,01 ONE FAMILY DWELLINGS,1,5488,48,NaN,A2,558 ELLSWORTH AVENUE,NaN,10465.0,1.0,0.0,1.0,"2,500",720.0,1935.0,1,A2,253500,2019-04-01
18244,3,OTHER,01 ONE FAMILY DWELLINGS,1,5936,31,NaN,A1,16 BAY RIDGE PARKWAY,NaN,11209.0,1.0,0.0,1.0,"2,880",2210.0,1925.0,1,A1,1300000,2019-04-01
18269,3,OTHER,01 ONE FAMILY DWELLINGS,1,3896,33,NaN,A5,15 VAN SICLEN COURT,NaN,11207.0,1.0,0.0,1.0,"1,974",1400.0,1910.0,1,A5,0,2019-04-01


In [30]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,5061.0,6808.683264,3907.147819,17.0,3973.0,6315.0,9708.0,16350.0
LOT,5061.0,72.994467,156.833411,1.0,21.0,41.0,68.0,2780.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,5061.0,11040.995653,501.017045,0.0,10465.0,11236.0,11413.0,11697.0
RESIDENTIAL_UNITS,5061.0,0.990911,0.096972,0.0,1.0,1.0,1.0,2.0
COMMERCIAL_UNITS,5061.0,0.022723,0.152960,0.0,0.0,0.0,0.0,2.0
TOTAL_UNITS,5061.0,1.013634,0.182245,0.0,1.0,1.0,1.0,3.0
GROSS_SQUARE_FEET,5061.0,1559.323849,740.282754,0.0,1168.0,1400.0,1775.0,12618.0
YEAR_BUILT,5061.0,1942.234539,37.761218,0.0,1925.0,1935.0,1955.0,2018.0
TAX_CLASS_AT_TIME_OF_SALE,5061.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [29]:
df.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,5061,1,01 ONE FAMILY DWELLINGS,5061,NaT,NaT
APARTMENT_NUMBER,2,2,RP.,1,NaT,NaT
TAX_CLASS_AT_PRESENT,5061,3,1,5013,NaT,NaT
BOROUGH,5061,5,4,2561,NaT,NaT
NEIGHBORHOOD,5061,9,OTHER,4727,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,5061,12,A1,1932,NaT,NaT
BUILDING_CLASS_AT_PRESENT,5061,16,A1,1930,NaT,NaT
SALE_DATE,5061,119,2019-01-31 00:00:00,96,2019-01-01,2019-04-30
LAND_SQUARE_FEET,5061,1469,"4,000",498,NaT,NaT
ADDRESS,5061,5001,277 NICHOLS AVENUE,4,NaT,NaT


## Do one-hot encoding of categorical features.

In [0]:
import category_encoders as cs

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['SALE_DATE', 'ZIP_CODE', 'ADDRESS']
features = train.columns.drop([target] + high_cardinality)

## Do feature selection with SelectKBest.

## Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

## Get mean absolute error for the test set.